# 集中趋势测度

本篇我们将讨论如何用单个数字来概括一组数据的方法，从而获取关于数据分布的信息。


#### 算数平均值

算术平均值经常被用于概括一组数值型数据的“平均水平”，定义为观测值之和除以观测值个数:

$$\mu = \frac{\sum_{i=1}^N X_i}{N}$$

其中 $X_1, X_2, \ldots , X_N$ 代表我们的观测值.

In [2]:
# 引入统计库
import scipy.stats as stats
import numpy as np

# 使用两组数据集为例
x1 = [1, 2, 2, 3, 4, 5, 5, 7]
x2 = x1 + [100]

print 'x1均值:', sum(x1), '/', len(x1), '=', np.mean(x1)
print 'x2均值:', sum(x2), '/', len(x2), '=', np.mean(x2)

x1均值: 29 / 8 = 3.625
x2均值: 129 / 9 = 14.3333333333


我们还可以定义一个<i>加权</i> 算术平均值，这对于明确指定每个观察值的计数次数很有用。例如在计算一个投资组合的平均价值时，更方便的说法是你70%的股票是XX类的，而不是列出你持有的每一只股票。加权平均值的定义为：

$$\sum_{i=1}^n w_i X_i $$

其中 $\sum_{i=1}^n w_i = 1$。事实上算术平均值是加权平均值的一个特例, 它使每个观测值的权重都等于 $w_i = 1/n$.

#### 中位数

中位数是将一组数据按递增或递减顺序排序时，出现在列表中间的数字。当列表有奇数𝑛个数据时,中位数是位置在第(𝑛+ 1)/ 2的数的值。当列表有偶数n个数据时，列表被分成两半，中间没有项目，所以中位数是位置在第𝑛/ 2和(𝑛+ 2)/ 2的两个数的值，取平均数。中位数受极值数据的影响小于算术平均值。它告诉我们的是将数据集一分为二的值，而无关乎其他值的大小。

In [3]:
print 'x1中位数:', np.median(x1)
print 'x2中位数:', np.median(x2)

Median of x1: 3.5
Median of x2: 4.0


#### 众数

众数是数据集中出现频率最高的值，与平均值和中位数不同，众数可以应用于非数值数据。对于相互独立的概率事件它也是适用的。例如，掷骰子的结果，频繁地投出6并不意味着接下来会投出5;所以知道数据集的众数是6比知道平均值是4.5更有用。

In [3]:
import pandas as pd
print pd.Series(x1).mode().tolist()

One mode of x1: 2
All of the modes of x1: [2, 5]


对于可以接受许多不同值的数据，例如日收益率数据，可能不会有任何值出现超过一次。对于这种情况，我们可以将值放入不同的区间范围中，类似构建直方图时所做的那样，每个值都对应着它所落入的那个区间范围，然后我们找到出现次数最频繁的区间作为众数。

In [4]:
# 获取标普的日收益率数据集并计算众数
start = '2017-01-01'
end = '2018-01-01'
pricing = get_pricing('AAPL', fields='price', start_date=start, end_date=end)
returns = pricing.pct_change()[1:]
print '收益率众数:', mode(returns.mode().tolist())

'''由于所有的收益率都是不同的，我们使用频率分布来获得另一种众数'''
# 将收益率按从低到高拆分成20个区间，np.histogram 返回区间的频率分布以及区间的边界范围
hist, bins = np.histogram(returns, 20) # 
maxfreq = max(hist)
# 找出所有箱体区间中，出现频率最高的那个
print '区间的众数:', [(bins[i], bins[i+1]) for i, j in enumerate(hist) if j == maxfreq]

Mode of returns: No mode
Mode of bins: [(-0.0012151920594887407, 0.0011015927020092131)]


#### 几何平均值

与算术平均采用加法不同，几何平均采用乘法:
$$ G = \sqrt[n]{X_1X_1\ldots X_n} $$

对于观测值$X_i \geq 0$，我们能够使用对数将公式改写成算数平均值的形式:
$$ \ln G = \frac{\sum_{i=1}^n \ln X_i}{n} $$

几何平均值总是$\geq$算数平均值（当观测值非负时），仅当所有观测值全部相同时，等号成立。

In [5]:
# Use scipy's gmean function to compute the geometric mean
print 'x1几何平均值:', stats.gmean(x1)
print 'x2几何平均值:', stats.gmean(x2)

Geometric mean of x1: 3.09410402498
Geometric mean of x2: 4.55253458762


如果我们想计算含有负数观测值的几何平均值呢? 拿资产日收益率为例来说（最小的日收益率是-1，即-100%），这个问题很容易解决。我们可以对日收益率数据集$R_t$加上1，得到$1 + R_t$, 这是资产连续两期之间的价格比率(与之相对的是价格之间的百分比变化, $R_t$). 它总是非负的. 因此我们能够计算出几何平均值,
$$ R_G = \sqrt[T]{(1 + R_1)\ldots (1 + R_T)} - 1$$

In [6]:
# 对收益率中的每个值加1，并计算几何平均值
ratios = returns + np.ones(len(returns))
R_G = stats.gmean(ratios) - 1
print '收益率的几何平均值:', R_G

Geometric mean of returns: 0.000463575391327


几何平均值被定义为，如果整个时间段内的收益率恒等于$R_G$, 由此计算得到的证券最终价格与前面举例的按照$R_1, \ldots, R_T$作为各期回报率计算得到的证券最终价格是一样的

In [7]:
T = len(returns)
init_price = pricing[0]
final_price = pricing[T]
print '初始价格:', init_price
print '最终价格:', final_price
print '按年化收益率R_G计算的最终价格:', init_price*(1 + R_G)**T

Initial price: 182.95
Final price: 205.52
Final price as computed with R_G: 205.52


#### 调和平均值
调和平均值不像其他类型的平均值使用的那么普遍，它定义为：
$$ H = \frac{n}{\sum_{i=1}^n \frac{1}{X_i}} $$
像几何平均值那样，我们可以将调和平均值的公式改写成算数平均值的形式。调和平均值的倒数是观测值倒数的算术平均值:
$$ \frac{1}{H} = \frac{\sum_{i=1}^n \frac{1}{X_i}}{n} $$
对于非负数$X_i$来说，调和平均值总是最大不会超过几何平均值(其最大不会超过算术平均值)，仅当所有的观测值相等时，它们才相等。

In [8]:
print 'Harmonic mean of x1:', stats.hmean(x1)
print 'Harmonic mean of x2:', stats.hmean(x2)

Harmonic mean of x1: 2.55902513328
Harmonic mean of x2: 2.86972365624


当数据能够按比率来自然地表达时，可以使用调和平均数。例如在平均成本策略中，定期使用固定数额的资金购买股票，股价越高，投资者买入的股票数量就越少。他们为股票支付的平均(算术平均数)金额是各期买入价格的调和平均数。

#### 点估计可能具有欺骗性
由于均值本质上，是将整个分布压缩成一个数值作为点估计或者说度量，其结果会隐藏了大量的信息。 你应该仔细确认在概括数据集的过程中没有丢失关键信息，并且你应该尽量避免在不考虑展布的情况下使用平均值（展布即数据的波动或发散程度，常见的表现形式有方差、标准差、四分位距，也叫内距）。

#### 潜在的分布可能是错误的

即使你正确测量了平均值和展布，如果数据分布不是你所认为的那样，那么这些指标也将没有意义。例如，使用标准差来测量事件的频率通常假设事件服从正态分布。如非必须不要假设分布，否则你应该严格检查数据是否符合你所假设的分布。